In [80]:
!pip install yfinance pandas numpy matplotlib scikit-learn ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [81]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR


In [82]:
# Function to fetch stock data
def fetch_stock_data(ticker, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date)

# Add technical indicators
def add_technical_indicators(df):
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['Volatility'] = df['Close'].rolling(window=20).std()
    df = df.dropna()
    return df

def train_model(data, model_type="Gradient Boosting Regressor"):
    data['Target'] = data['Close'].shift(-1)
    data = data.dropna()
    X = data[['SMA_20', 'EMA_20', 'Volatility']]
    y = data['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Select model based on user input
    if model_type == "Gradient Boosting Regressor":
        model = GradientBoostingRegressor(random_state=42, n_estimators=100, learning_rate=0.1)
    elif model_type == "Random Forest Regressor":
        model = RandomForestRegressor(random_state=42, n_estimators=100)
    elif model_type == "Linear Regression":
        model = LinearRegression()
    elif model_type == "Support Vector Regressor":
        model = SVR(kernel='rbf', C=100, gamma=0.1)
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    return X_test, y_test, predictions, model



# Updated function to plot predictions
def plot_predictions(y_test, predictions):
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, predictions, alpha=0.6, label='Predicted vs Actual', color='orange')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', label='Perfect Prediction')
    plt.xlabel("Actual Values", fontsize=12)
    plt.ylabel("Predicted Values", fontsize=12)
    plt.title("Predicted vs Actual Values", fontsize=14)
    plt.legend(fontsize=10)
    plt.show()



In [83]:
# Create widgets for interactivity
ticker_widget = widgets.Text(value='AAPL', description='Ticker:')
start_date_widget = widgets.DatePicker(description='Start Date', value=pd.to_datetime('2022-01-01'))
end_date_widget = widgets.DatePicker(description='End Date', value=pd.to_datetime('2023-01-01'))
button = widgets.Button(description="Fetch and Predict")
# New widgets for additional prediction options
future_date_widget = widgets.DatePicker(description="Future Date")
predict_next_day_button = widgets.Button(description="Predict Next Day")
predict_future_date_button = widgets.Button(description="Predict Future Date")

In [84]:
def on_button_click(b):
    clear_output(wait=True)
    display(ticker_widget, start_date_widget, end_date_widget, button, predict_next_day_button, future_date_widget, predict_future_date_button, model_selector)
    
    ticker = ticker_widget.value
    start_date = start_date_widget.value
    end_date = end_date_widget.value
    model_type = model_selector.value
    
    # Fetch data
    data = fetch_stock_data(ticker, start_date, end_date)
    if data.empty:
        print("No data found for the given inputs. Please check the ticker or date range.")
        return
    
    # Add technical indicators
    data = add_technical_indicators(data)
    
    # Train model and predict
    X_test, y_test, predictions, model = train_model(data, model_type)
    print(f"Model: {model_type}")
    print("Model Evaluation Metrics:")
    print(f"MAE: {mean_absolute_error(y_test, predictions):.2f}")
    print(f"MSE: {mean_squared_error(y_test, predictions):.2f}")
    print(f"R² Score: {r2_score(y_test, predictions):.2f}")
    
    # Plot predictions
    plot_predictions(y_test, predictions)


In [85]:
def predict_next_day(b):
    ticker = ticker_widget.value
    start_date = start_date_widget.value
    end_date = end_date_widget.value
    model_type = model_selector.value
    
    # Fetch data
    data = fetch_stock_data(ticker, start_date, end_date)
    if data.empty:
        print("No data found for the given inputs.")
        return
    
    # Add indicators and train model
    data = add_technical_indicators(data)
    _, _, _, model = train_model(data, model_type)
    
    # Predict next day's price
    last_row = data[['SMA_20', 'EMA_20', 'Volatility']].iloc[-1:]
    next_day_prediction = model.predict(last_row)
    print(f"Model: {model_type}")
    print(f"Predicted closing price for the next day: {next_day_prediction[0]:.2f}")


def predict_future_date(b):
    ticker = ticker_widget.value
    start_date = start_date_widget.value
    end_date = end_date_widget.value
    future_date = future_date_widget.value
    model_type = model_selector.value

    # Fetch data
    data = fetch_stock_data(ticker, start_date, end_date)
    if data.empty:
        print("No data found for the given inputs.")
        return

    # Add technical indicators
    data = add_technical_indicators(data)

    # Ensure the index is a DatetimeIndex
    if not isinstance(data.index, pd.DatetimeIndex):
        data.index = pd.to_datetime(data.index)

    # Train the selected model
    _, _, _, model = train_model(data, model_type)

    # Validate future_date and last_date
    try:
        future_date = pd.Timestamp(future_date)
        last_date = data.index[-1]
        print(f"Future Date: {future_date}, Last Date in Dataset: {last_date}")
    except Exception as e:
        print(f"Error with dates: {e}")
        return

    # Calculate num_days_ahead
    try:
        num_days_ahead = int((future_date - last_date).days)
        if num_days_ahead <= 0:
            print("Future date must be beyond the last available date in the dataset.")
            return
    except Exception as e:
        print(f"Error calculating days ahead: {e}")
        return

    # Iterative predictions for the future date
    current_row = data[['SMA_20', 'EMA_20', 'Volatility']].iloc[-1:].copy()
    next_prediction = None
    for _ in range(num_days_ahead):
        next_prediction = model.predict(current_row)
        current_row = pd.DataFrame(
            [[next_prediction[0], current_row['EMA_20'].iloc[0], current_row['Volatility'].iloc[0]]],
            columns=['SMA_20', 'EMA_20', 'Volatility']
        )

    print(f"Model: {model_type}")
    print(f"Predicted closing price for {future_date.date()}: {next_prediction[0]:.2f}")


In [86]:
button.on_click(on_button_click)
predict_next_day_button.on_click(predict_next_day)
predict_future_date_button.on_click(predict_future_date)

In [88]:
from ipywidgets import VBox, HBox

# Arrange widgets into groups
layout = VBox([
    VBox([
        HBox([ticker_widget, model_selector]),
        HBox([start_date_widget, end_date_widget]),
        HBox([button, predict_next_day_button, predict_future_date_button])
    ]),
    future_date_widget  # Future date prediction input
])

# Display the layout
display(layout)
